In [8]:
import pandas as pd
import numpy as np
import requests
import http.client, urllib.parse
from math import cos, asin, sqrt, pi
import datetime as datetime

In [2]:

geo_api_key = #Go get an api key from positionstack.com

In [18]:
def find_lats(city, state):
    
    conn = http.client.HTTPConnection('api.positionstack.com')

    params = urllib.parse.urlencode({
        'access_key': geo_api_key,
        'query': city,
        'region': state,
        'limit': 1,
        })

    conn.request('GET', '/v1/forward?{}'.format(params))

    res = conn.getresponse()
    data = res.read().decode('utf-8')

    latitude = float(data.split('{')[2].split(',')[0].split(':')[1])
    longitude = float(data.split('{')[2].split(',')[1].split(':')[1])
    lat_lon = [latitude,longitude]
    
    return lat_lon

In [58]:
def reverse(city, state):
    now = datetime.datetime.now()
    five_pm = datetime.datetime(now.year,now.month,now.day,17,0,0)
    day_pct = (five_pm-now).seconds/(24*60*60)
    
    lat_lon = find_lats(city, state)
    new_long = lat_lon[1]+360*day_pct
    
    conn = http.client.HTTPConnection('api.positionstack.com')

    params = urllib.parse.urlencode({
        'access_key': geo_api_key,
        'query': str(lat_lon[0])+','+str(new_long),
        'limit': 1,
        })

    conn.request('GET', '/v1/reverse?{}'.format(params))

    res = conn.getresponse()
    data = res.read().decode('utf-8')
    town = data.split('{')[2].split(',')[4].split(':')[1]
    country = data.split('{')[2].split(',')[15].split(':')[1]
    print(town, country)
    print(lat_lon[0], new_long)
    

In [60]:
reverse('Brooklyn', 'New York')

"Tyrrhenian Sea" null
40.652256 10.843418


In [59]:
reverse('Atlanta', 'Georgia')

"El Bayadh" "Algeria"
33.769805 0.4395856666666731
